In [1]:
# !pip install transformers

The !pip install transformers installs the pre compiled transformers package (.whl file) from pypi

import transformers
transformers.__version__

Prints ---> 4.27.4

Whereas the !pip install git+https://github.com/huggingface/transformers.git , check-out the latest version from git repository and compiles it on the google colab environment and then installs the package.
It installs the latest development version.

import transformers
transformers.__version__

Prints ---> 4.28.0.dev0

In [2]:
!pip install git+https://github.com/huggingface/transformers.git
# !pip install -q tensorflow
#root

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-y4jg8t7e
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-y4jg8t7e
  Resolved https://github.com/huggingface/transformers.git to commit 4f58fc9c823c43b67a6ce1c44be28f8aecc3c8d9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.52.0.dev0-py3-none-any.whl size=11447297 sha256=809449496cbfd8588f417af5e1b5f04d8205c748e48f2ad25b1a06872b273276
  Stored in directory: /tmp/pip-ephem-wheel-cache-kti0tejt/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


A tokenizer that turns input text into tokens understandable by the model.

A tokenizer is the bridge between raw text (what humans read) and model input (what machines understand).
It breaks down sentences into small pieces called tokens that the model can process.

Text → Tokenizer → Tokens → Model

Model Output → Tokens → Tokenizer → Human-readable Text

Important parameters used in text generation:

* do_sample=True: Enables sampling to make the output creative.

  Input: "Once upon a time,"

  Output 1: "Once upon a time, a dragon flew over the hills."

  Output 2: "Once upon a time, a lonely knight wandered the desert."

  (Different outputs each time)

* max_length: Controls how long the generated text can be.

  max_length=30

  Output: A short paragraph.

  max_length=100

  Output: A much longer, story-like paragraph.

* top_k: sampling limits the model to only considering the top-k highest probability tokens when picking the next word.

  If top_k=50, the model picks randomly among the 50 most likely next words.

  If top_k=0, no restriction: it considers the full vocabulary (full randomness).

* temperature: controls the creativity of the model (i.e., randomness); lower values make output more focused.

  temperature=0.2

  Input: "The cat sat" Output: "The cat sat on the mat and fell asleep."

  temperature=1.3

  Input: "The cat sat" Output: "The cat sat weaving symphonies into golden dreams across universes."

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import tensorflow as tf
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Replace with the actual model path (adjust if you're not in Colab)
model_path = "/content/drive/MyDrive/HolmesModel"

# Load fine-tuned tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

import torch

# Set device properly for PyTorch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def generate_response(input_text, max_length):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    output = model.generate(
        input_ids=input_ids,
        do_sample=True,
        max_length=max_length,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    response_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return response_text


### Install Flask-ngrok

In [5]:
!pip install pyngrok


In [6]:
from pyngrok import ngrok

# Replace with your ngrok auth token
!ngrok config add-authtoken 2vpKSeY2c2BbKT07CrCnqf3YOhm_5BrVgTTEvvbmBRSo18pGi
#ngrok.set_auth_token("2vpKSeY2c2BbKT07CrCnqf3YOhm_5BrVgTTEvvbmBRSo18pGi")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


### Start the Flask Server

Copy the generated ngrok server url and use in the client code

In [7]:
def clean_response(raw_response, query):
    """
    Cleans the raw GPT-2 output to make it more suitable for a chatbot interface.

    Args:
        raw_response (str): The raw text generated by GPT-2
        query (str): The original user query (for context-aware cleaning)

    Returns:
        str: Cleaned, more natural sounding response
    """
    # 1. Remove the original query if it's repeated at the start
    if raw_response.startswith(query):
        cleaned = raw_response[len(query):].lstrip(",.!? \n")
    else:
        cleaned = raw_response

    # 2. Trim everything after the last complete sentence
    last_punctuation = max(
        cleaned.rfind("."),
        cleaned.rfind("!"),
        cleaned.rfind("?"),
        cleaned.rfind("\n")
    )

    if last_punctuation > 0:
        cleaned = cleaned[:last_punctuation + 1]

    # 3. Remove any repetitive patterns (common in GPT-2 outputs)
    sentences = [s.strip() for s in cleaned.split(". ") if s.strip()]
    if len(sentences) > 1 and sentences[-1] in sentences[:-1]:
        cleaned = ". ".join(sentences[:-1]) + "."

    # 4. Fix common formatting issues
    cleaned = cleaned.replace("  ", " ")  # Double spaces
    cleaned = cleaned.replace("\n", " ")  # Newlines
    cleaned = cleaned.replace('""', '"')  # Double quotes

    # 5. Capitalize first letter and ensure ends with punctuation
    cleaned = cleaned.strip()
    if cleaned:
        cleaned = cleaned[0].upper() + cleaned[1:]
        if cleaned[-1] not in {".", "!", "?"}:
            cleaned += "."

    # 6. Specific cleaning for Sherlock persona
    if "Sherlock" in query or "Holmes" in query:
        sherlock_signatures = [
            " Elementary, my dear Watson.",
            " As I often tell Watson...",
            " The game is afoot!"
        ]
        if not any(sig in cleaned for sig in sherlock_signatures):
            cleaned += sherlock_signatures[0]

    return cleaned

In [8]:
!pip install flask-cors

In [10]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from flask_cors import CORS
from datetime import datetime  # Add this import
import logging  # For better error handling

# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = Flask(__name__)

# Configure CORS with additional security options
CORS(app, resources={
    r"/chatbot": {
        "origins": ["*"],  # In production, replace with your frontend URL
        "methods": ["POST", "OPTIONS"],
        "allow_headers": ["Content-Type"]
    },
    r"/health": {
        "origins": "*",
        "methods": ["GET"]
    }
})

@app.route('/health', methods=['GET'])
def health():
    response = {'text': 'Success', 'timestamp': datetime.now().isoformat()}
    return jsonify(response)

@app.route('/chatbot', methods=['POST'])
def chatbot():
    try:
        # Validate input
        if not request.is_json:
            return jsonify({'error': 'Request must be JSON'}), 400

        input_text = request.json.get('text')
        if not input_text or not input_text.strip():
            return jsonify({'error': 'Text cannot be empty'}), 400

        # Generate and clean response
        raw_response = generate_response(input_text, max_length=50)
        cleaned_text = clean_response(raw_response, input_text)

        response = {
            'raw_response': raw_response,
            'cleaned_response': cleaned_text,
            'timestamp': datetime.now().isoformat()
        }
        return jsonify(response)

    except Exception as e:
        logger.error(f"Error in chatbot endpoint: {str(e)}")
        return jsonify({'error': 'Internal server error'}), 500

if __name__ == '__main__':
    port = 5000
    try:
        public_url = ngrok.connect(port).public_url
        print(f" * ngrok tunnel: {public_url}")
        app.run(port=port)
    except Exception as e:
        logger.error(f"Failed to start server: {str(e)}")
        raise

 * ngrok tunnel: https://ab49-34-45-116-75.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 19:43:27] "OPTIONS /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 19:43:32] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 19:44:18] "GET /health HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 19:44:33] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 19:46:13] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 19:47:01] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 19:47:38] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 19:49:11] "OPTIONS /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 19:49:14] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Apr/2025 19:49:43] "OP